# Word2vec

На этом занятии мы посмотрим, как работать с заранее натренированными word embeddings в библиотеке gensim, и научимся использовать вектора в решении реальной задачи.

Загрузим вектора, полученные на корпусе GoogleNews:

In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

Создадим объект, с которым будем работать (это займёт несколько минут):

In [ ]:
from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

Теперь посмотрим, какую информацию мы можем получать с помощью этой библиотеки.

1. Получать векторное представление слова (попробуйте также любое другое слово):

In [ ]:
word2vec['sun']

2. Находить n самых близких слов к заданному (по дефолту n=10, Вы можете изменить его значение):

In [ ]:
word2vec.similar_by_word('student')

3. Считать близость между словами или списками слов (чем значение выше, тем слова семантически ближе друг к другу):

In [ ]:
print(word2vec.similarity('sun', 'moon'))
print(word2vec.similarity('sun', 'car'))

4. И некоторые другие вещи, о которых можно почитать в [документации библиотеки](https://pypi.org/project/gensim/).

Теперь попробуем использовать векторные представления слов на практике. Задача, которую мы будем решать - определение тематики текстов (Topic Identification).

Для этого мы воспользуемся корпусом новостных текстов BBC (из текстов взяты только заголовок и краткое описание статьи), которым приписаны категории.

Загрузим корпус:

In [ ]:
! wget https://raw.githubusercontent.com/BruchesLena/DataSets/master/bbc_texts.txt

Распарсим файл, в итоге у нас будет словарь, где ключ - новостной текст, значение - категория этого текста:

In [ ]:
data = {}
with open('bbc_texts.txt', 'r') as f:
  for line in f.read().split('\n'):
    parts = line.split(' ;')
    data[parts[0]] = parts[1]
    
print(len(data))
tags = list(set(data.values()))
print(tags)

tag2idx = {t: i for i, t in enumerate(tags)}
print(tag2idx)

Давайте посмотрим, как выглядят наши данные. Для этого выведим на экран 5 первых samples:

In [ ]:
n_samples = 5
i = 0
for text, label in data.items():
  print(f'{text} -> {label}')
  i += 1
  if i == n_samples:
    break

Можно заметить, что все 5 первых текстов относятся к категории *business*. Действительно в корпусе все тексты упорядочены по тематикам, что плохо для обучения алгоритма. Давайте перемешаем данные:

In [ ]:
import random

l = list(map(tuple, data.items()))
random.shuffle(l)
data = dict(l)

Посмотрим на первые 5 примеров теперь:

In [ ]:
n_samples = 5
i = 0
for text, label in data.items():
  print(f'{text} -> {label}')
  i += 1
  if i == n_samples:
    break

Теперь нам нужно текст каждой новости представить в виде вектора. Традиционно существуют 2 способа:
1. Сложить вектора всех слов в тексте;
2. Усреднить вектора всех слов в тексте.

Мы попробуем оба варианта, чтобы посмотреть, какой лучше работает на практике.

Начнём со сложения векторов.

Напишите функцию, которая принимает на вход текст, а возвращает его векторное представление (сумму векторов всех слов в тексте). Для простоты можете текст разделять на слова по пробелам.

In [ ]:
import numpy as np

def vectorize_as_sum(text):
  pass #Ваш код
  
  return vector

Давайте с помощью этой функции векторизуем все примеры из нашего датасета:

In [ ]:
X = [] # векторизованные тексты
y = [] # категории, которые будем предсказывать

for text, label in data.items():
  vector = vectorize_as_sum(text)
  X.append(vector)
  targets = np.zeros(len(tags))
  targets[tag2idx[label]] = 1.0
  y.append(targets)
  
X = np.array(X)
y = np.array(y)

Убедимся, что мы всё сделали правильно:


In [ ]:
print(X.shape == (2105, 300))
print(y.shape == (2105, 5))

Разделим наши данные на тренировочные и тестовые:

In [ ]:
X_train, y_train = X[:2000], y[:2000]
X_test, y_test = X[2000:2100], y[2000:2100]

Теперь создадим простую модель, которую будем обучать, и запустим её. Наша модель будет состоять только из простых полносвязных слоёв Dense. 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

model = Sequential() # создаём "контейнер", в который будем складывать слои нашей нейронной сети
model.add(Dense(300, input_dim=(300), activation='relu')) # первый слой, в котором обязательно указываем размерность входного вектора; содержит 300 нейронов и функцию активации relu
model.add(Dense(50, activation='relu')) # скрытый слой, который содержит 50 нейронов и функцию активации relu
model.add(Dense(len(tags), activation='softmax')) # выходной слой, который содержит количествр нейронов, равное количеству наших классов (категорий), и функцию активации softmax

model.summary() # посмотрим на количество параметров нашей сети

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]) # указываем, что сеть обучается с помощью алгоритма adam, в качестве функции потери мы используем кросс-энтропию

saver = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True,mode='auto') # указываем, что сохраням модель с лучшими показателями
stopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto') # указываем, когда останавливаем обучение (после того, как значение ошибки на тестовых данных не уменьшается в течение 3 эпох), чтобы предотвратить переобучение

model.fit(X_train, y_train, batch_size=24, epochs=20, validation_data=(X_test, y_test),
                     verbose=2, callbacks=[saver, stopper]) # передаём данные для обучения и тестирования, размер батча, количество эпох для обучения

Попробуйте поизменять настройки этой сети, а именно:
* количество скрытых слоёв;
* количество нейронов на скрытых слоях;
* размер батча;
* количество эпох (в т.ч.,  в EarlyStopping).

В итоге у вас должна получиться точность на тестовых данных не менее 0.96.

После того, как вы получили точность более 0.96, давайте проверим модель на реальных данных. Посмотрите, что предсказывает модель на других текстах (вставьте из тексты из реальных новостей/придумайте свои):

In [ ]:
text = 'Apple introduced new iPhone'
vector = vectorize_as_sum(text)
vector = [vector]
vector = np.array(vector)
prediction = model.predict(vector)[0]
print(prediction)

predicted_label = np.argmax(prediction)
print('Category:', tags[predicted_label])

Отлично! Теперь давайте проделаем всё то же самое, но текст будем представлять как усреднённые вектора слов этого текста. Напишите функцию, которая таким образом векторизует текст:

In [ ]:
def vectorize_as_avg(text):
  pass #Ваш код
  
  return vector


In [ ]:
X = [] # векторизованные тексты
y = [] # категории, которые будем предсказывать

for text, label in data.items():
  vector = vectorize_as_avg(text)
  X.append(vector)
  targets = np.zeros(len(tags))
  targets[tag2idx[label]] = 1.0
  y.append(targets)
  
X = np.array(X)
y = np.array(y)

In [ ]:
print(X.shape == (2105, 300))
print(y.shape == (2105, 5))

In [ ]:
X_train, y_train = X[:2000], y[:2000]
X_test, y_test = X[2000:2100], y[2000:2100]

Ваша задача такая же: получить точность на тестовых данных не менее 0.96.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

model = Sequential()
model.add(Dense(300, input_dim=(300), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(tags), activation='softmax'))

model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

saver = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True,mode='auto')
stopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

model.fit(X_train, y_train, batch_size=24, epochs=20, validation_data=(X_test, y_test),
                     verbose=2, callbacks=[saver, stopper])

Проверяем модель:

In [ ]:
text = 'Apple introduced new iPhone'
vector = vectorize_as_avg(text)
vector = [vector]
vector = np.array(vector)
prediction = model.predict(vector)[0]
print(prediction)

predicted_label = np.argmax(prediction)
print('Category:', tags[predicted_label])